Now that I've streamlined the MCMC process, I am going to submit multiple chains simultaneously. This notebook will make multiple, similar config files, for broad comparison. 

This may be rolled into pearce as a helper function, I haven't decided.

Update from the older SLAC notebook, which is now outdated

For rmin 0, 0.5, 1.0:

    For no ab, HSAB and CorrAB emu:

        Vpeak sham
        Mpeak sham
        HOD
        HSAB HOD 

In [19]:
import yaml
import copy
from os import path
import numpy as np
import h5py

In [20]:
emu_path = '/u/ki/swmclau2/des/'
#emu_path = '~/scratch'

In [21]:
cov_path = '/u/ki/swmclau2/Git/pearce/bin/'
# cov_path = '~/Git/pearce/bin'

In [22]:
orig_cfg_fname = path.join(cov_path, 'mcmc/nh_gg_sham_hsab_mcmc_config.yaml')
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [23]:
bsub_template="""#BSUB -q medium 
#BSUB -W 8:00
#BSUB -J {jobname}
#BSUB -oo /u/ki/swmclau2/Git/pearce/bin/mcmc/config/{jobname}.out    
#BSUB -n 8 
#BSUB -R "span[ptile=8]"

python /u/ki/swmclau2/Git/pearce/pearce/inference/initialize_mcmc.py {jobname}.yaml
python /u/ki/swmclau2/Git/pearce/pearce/inference/run_mcmc.py {jobname}.yaml
"""

In [24]:
import numpy as np
r_bins = np.logspace(-0.3, 2, 21)
print r_bins

[  0.50118723   0.65313055   0.85113804   1.10917482   1.44543977
   1.88364909   2.45470892   3.19889511   4.16869383   5.43250331
   7.07945784   9.22571427  12.02264435  15.6675107   20.41737945
  26.6072506   34.67368505  45.18559444  58.88436554  76.73614894
 100.        ]


In [25]:
emu_names = ['VDF', 'Xi', 'VDF_Xi']
emu_fnames = [[path.join(emu_path, 'Aemulators/vdf_zheng07/PearceVDFCosmo.hdf5')],
               [path.join(emu_path, 'Aemulators/xi_zheng07/PearceXiCosmo.hdf5')],
             [path.join(emu_path, 'Aemulators/vdf_zheng07/PearceVDFCosmo.hdf5'),
               path.join(emu_path, 'Aemulators/xi_zheng07/PearceXiCosmo.hdf5')]]


In [26]:
obs_names = ['vdf', 'xi']

data_fnames = [path.join(cov_path, 'mock_measurements/testbox_1_1_%s.npy'%o)
                for o in obs_names]

data_fnames.append(copy.copy(data_fnames))

In [27]:
data_fnames

['/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_1_1_vdf.npy',
 '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_1_1_xi.npy',
 ['/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_1_1_vdf.npy',
  '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_1_1_xi.npy']]

In [28]:
meas_cov_fnames = []
emu_cov_fnames = []
for obs, emu, data in zip(obs_names, emu_fnames, data_fnames ):
    #if obs =='vdf':
    #    continue
    #with h5py.File(emu[0], 'r') as f:
    #    rel_meas_cov = f.attrs['rel_cov']
    #    rel_emu_cov = f.attrs['rel_emu_cov']
    
    # get rid of nans 
    #rel_meas_cov[np.isnan(rel_meas_cov)] = 1e-9
    #o = np.load(data)
    #meas_cov = rel_meas_cov*np.outer(o, o)    
    #emu_cov = rel_emu_cov*np.outer(o,o)
    
    meas_cov_fname = path.join(cov_path, 'covmat/%s_sim_covmat.npy'%obs)

    emu_cov_fname = path.join(cov_path, 'optimization/%s_emu_cov_lpw.npy'%obs)
    
    meas_cov_fnames.append(meas_cov_fname)
    emu_cov_fnames.append([emu_cov_fname])
    

In [29]:
emu_cov_fnames.append([e[0] for e in emu_cov_fnames])

In [30]:
from scipy.linalg import block_diag
mc = block_diag(*[np.load(fname) for fname in meas_cov_fnames])
cov_fname = path.join(cov_path, 'covmat/%s_full_covmat.npy'%('_'.join(obs_names) ))
np.save(cov_fname, mc)
meas_cov_fnames.append(cov_fname)

In [31]:
obs_names.append(copy.copy(obs_names))

In [32]:
%matplotlib inline
from matplotlib import pyplot as plt

In [33]:
def cov_to_corr(cov):
    std = np.sqrt(np.diag(cov))
    
    denom = np.outer(std, std)
    
    return cov/denom

In [34]:
n_walkers = 250
n_steps = 50000

## Vpeak SHAM
----


## NFWized SHAM
----


## Shuffled SHAM
----


## Universe Machine
----


## HOD
----


In [35]:
#orig_cfg_fname = '/u/ki/swmclau2//Git/pearce/bin/mcmc/nh_gg_sham_hsab_mcmc_config.yaml'
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [37]:
tmp_cfg = copy.deepcopy(orig_cfg)

directory = path.join(cov_path, "mcmc/config/")
output_dir = path.join(emu_path, "PearceMCMC/")

jobname_template = "HOD_1_1_rmin_{rmin}_{emu_name}"#_fixed_HOD"

for rmin in [0.1, 1.0, 5.0]:
    for emu_fname, emu_name, emu_cov, obs, meas_cov, data_name in\
    zip(emu_fnames, emu_names, emu_cov_fnames, obs_names, meas_cov_fnames,\
        data_fnames):
        
        #if rmin is not None:
        tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}#, 'cosmo':0}
        #else:
        #    tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin, 'csomo':0}

            
        tmp_cfg['emu']['training_file'] = emu_fname
        tmp_cfg['emu']['emu_type'] = ['LemonPepperWet' for i in xrange(len(emu_fname))]
        tmp_cfg['emu']['emu_cov_fname'] = emu_cov
        
        tmp_cfg['data']['obs']['obs'] = [obs] if type(obs) is not list else obs
        tmp_cfg['data']['obs']['rbins'] = list(r_bins)

        tmp_cfg['data']['true_data_fname']= [data_name] if type(data_name) is not list else data_name
        tmp_cfg['data']['true_cov_fname'] = meas_cov
        
        
        tmp_cfg['data']['cov']['meas_cov_fname'] = meas_cov_fname#tmp_cfg['data']['true_cov_fname']
        tmp_cfg['data']['cov']['emu_cov_fname'] = tmp_cfg['emu']['emu_cov_fname'] # TODO make this not be redundant
        
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')
        
        tmp_cfg['sim']= {'gal_type': 'HOD',
       'hod_name': 'zheng07',
       'hod_params': {'alpha': 1.0,
        'logM0': 13.0,
        'logM1': 14.0,
        'sigma_logM': 0.2,
        'conc_gal_bias': 1.0},
       'nd': '5e-4',
       'scale_factor': 1.0,
       'min_ptcl': 100, 
       'sim_hps': {'boxno': 1,
        'realization': 0,
        'system': 'ki-ls'},
       'simname': 'trainingbox'}
        
        # TODO i shouldnt have to specify this this way
        tmp_cfg['data']['sim'] = tmp_cfg['sim']
        
        tmp_cfg['chain']['nwalkers'] = n_walkers
        tmp_cfg['chain']['nsteps'] = n_steps
        tmp_cfg['chain']['mcmc_type'] = 'normal'
        
        # fix params during MCMC    
        #tmp_cfg['chain']['fixed_params'].update(tmp_cfg['sim']['hod_params'])
        
        #try:
        #    del tmp_cfg['data']['true_data_fname']
        #    del tmp_cfg['data']['true_cov_fname']
        #except KeyError:
        #    pass

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        #with open(path.join(directory, jobname + '.sbatch'), 'w') as f:
        #    f.write(sbatch_template.format(jobname=jobname))
            
        with open(path.join(directory, jobname + '.bsub'), 'w') as f:
            f.write(bsub_template.format(jobname=jobname))
            #f.write(bsub_restart_template.format(jobname=jobname))

## HSAB HOD
----
